# Projet ARF : inpainting 
##### TREÜ Marc ; KARMIM Yannis

### Préambule : régression linéaire, régression ridge et LASSO

In [121]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression,Ridge, RidgeCV, Lasso, LassoCV
from sklearn import preprocessing

In [122]:
# Chargement des données USPS. 

def load_usps(filename):
    with open(filename,"r") as f:
        f.readline()
        data =[ [float(x) for x in l.split()] for l in f if len(l.split())>2]
    tmp = np.array(data)
    return tmp[:,1:],tmp[:,0].astype(int)

In [123]:
train = load_usps('data/USPS_train.txt')
trainX, trainY = train
test = load_usps('data/USPS_test.txt')
testX, testY = test

On fait une fonction qui permet de séléctionner deux classes dans USPS, afin de faire de la classification à l'aide de la régression linéaire. 

#### Regression Linéaire

In [124]:
def fit_linear_regression(n1,n2,dataX,dataY):
    
    """Compare deux classes représentant les nombres n1 et n2
    en utilisant une régression linéaire ."""
    
    X = dataX[np.where(np.in1d(dataY, [n1,n2]))]
    Y = dataY[np.where(np.in1d(dataY, [n1,n2]))]
    
    
    
    return LinearRegression().fit(X,Y)

In [125]:
# Choix des classes : 
n1 = 4
n2 = 7

reg = compare_classe(n1,n2,trainX,trainY)

In [126]:
reg.score(testX[np.where(np.in1d(testY, [n1,n2]))],testY[np.where(np.in1d(testY, [n1,n2]))])

0.7972470735084725

#### Ridge Regression ( sans Cross Val ) 

In [127]:
def fit_ridge_regression(n1,n2,dataX,dataY,alpha=1.0):
    """Compare deux classes représentant les nombres n1 et n2
    en utilisant une Ridge Regression."""
    
    X = dataX[np.where(np.in1d(dataY, [n1,n2]))]
    Y = dataY[np.where(np.in1d(dataY, [n1,n2]))]
    
    
    
    return Ridge(alpha=alpha).fit(X,Y)

In [128]:

ridge = fit_ridge_regression(n1,n2,trainX,trainY,alpha=1.0)

In [129]:
ridge.score(testX[np.where(np.in1d(testY, [n1,n2]))],testY[np.where(np.in1d(testY, [n1,n2]))])

0.8140303534137986

#### Ridge Regression ( avec Cross Val )

In [130]:
def fit_ridge_regressionCV(n1,n2,dataX,dataY,nb_cv = 10):
    """Compare deux classes représentant les nombres n1 et n2
    en utilisant une Ridge Regression avec Cross Val."""
    
    X = dataX[np.where(np.in1d(dataY, [n1,n2]))]
    Y = dataY[np.where(np.in1d(dataY, [n1,n2]))]
    clf = RidgeCV(cv = nb_cv)
    clf.fit(X,Y)
    print("Meilleur alpha trouvé par Cross Validation : ",clf.alpha_)
    
    return clf 

In [131]:
ridgeCV = fit_ridge_regressionCV(n1,n2,trainX,trainY,nb_cv=10)


Meilleur alpha trouvé par Cross Validation :  10.0


In [132]:
ridgeCV.score(testX[np.where(np.in1d(testY, [n1,n2]))],testY[np.where(np.in1d(testY, [n1,n2]))])

0.8225618454000782

#### LASSO 

In [138]:
def fit_LASSO(n1,n2,dataX,dataY,alpha = 10):
    """Compare deux classes représentant les nombres n1 et n2
    en utilisant une Ridge Regression avec Cross Val."""
    
    X = dataX[np.where(np.in1d(dataY, [n1,n2]))]
    Y = dataY[np.where(np.in1d(dataY, [n1,n2]))]
    clf = Lasso(alpha = alpha)
    clf.fit(X,Y)
    
    
    return clf 

In [143]:
lasso = fit_LASSO(n1,n2,trainX,trainY,alpha=0.1)

In [144]:
lasso.score(testX[np.where(np.in1d(testY, [n1,n2]))],testY[np.where(np.in1d(testY, [n1,n2]))])

0.7722916806139399

In [148]:
def fit_LASSO_CV(n1,n2,dataX,dataY,nb_cv = 10):
    """Compare deux classes représentant les nombres n1 et n2
    en utilisant une Ridge Regression avec Cross Val."""
    
    X = dataX[np.where(np.in1d(dataY, [n1,n2]))]
    Y = dataY[np.where(np.in1d(dataY, [n1,n2]))]
    clf = LassoCV(cv = nb_cv)
    clf.fit(X,Y)
    print("Meilleur alpha trouvé par Cross Validation : ",clf.alpha_)
    
    return clf 

In [149]:
lassoCV = fit_LASSO_CV(n1,n2,trainX,trainY)

Meilleur alpha trouvé par Cross Validation :  0.007311354380425852


In [150]:
lassoCV.score(testX[np.where(np.in1d(testY, [n1,n2]))],testY[np.where(np.in1d(testY, [n1,n2]))])

0.8235462778648425

En bonus faire une matrice de confusion pour les trois modèles


### LASSO et Inpainting